## 종만북 chapter 26. 트라이

정수나 실수형 변수는 그 크기가 정해져 있어 비교에 상수 시간이 걸린다고 봐도 되지만,  
**문자열 변수를 비교하는 데는 최악의 경우 문자열의 길이에 비례하는 시간이 걸린다.**  
따라서 정수나 실수 키에 대해서 성능이 좋은 탐색 자료구조들도 문자열을 키로 사용할 때 시간이 오래 걸릴 수 있다.  
N개의 원소를 갖는 이진 검색 트리에서 원하는 원소를 찾으려면 $O(logN)$번의 비교를 해야하는데,  
문자열의 비교에는 그 길이에 비례하는 시간이 걸리므로 문자열의 최대 길이 M을 곱한 $O(MlogN)$이 최종 시간 복잡도가 된다.  
  
이런 문제를 해결하기 위해 고안된 문자열 특화 자료 구조의 대표적인 예로 **트라이(Trie)**가 있다.  
트라이는 문자열의 집합을 표현하는 트리 자료 구조로, 집합 내에서 원하는 원소를 찾는 작업을 O(M)으로 할 수 있다.  
- 트라이는 집합에 포함된 문자열의 접두사들에 대응되는 노드들이 서로 연결된 트리이다.  
- 한 접두사의 맨 뒤에 글자를 덧붙여 다른 접두사를 얻을 수 있을 때 두 노드는 부모 자식 관계로 연결된다.  
- 두 노드를 연결하는 간선은 덧붙인 글자에 대응된다.  
- 루트노드는 항상 길이 0인 문자열에 대응되며, 노드의 깊이가 깊어질 때마다 대응되는 문자열의 길이가 1씩 늘어난다.   
  
탐색을 뜻하는 단어 'Retrieval'에서 유래


문자열 집합 S = {"BE", "BET", "BUS", "TEA", "TEN"}를 저장해보자.  
<img src="https://github.com/hyperpace/study_algo_gangnam/blob/master/%EB%B0%95%EC%83%81%EC%9A%B0/imgs/KakaoTalk_20190508_192319889.jpg?raw=true" width="600">

노랗게 칠한 노드들은 종료 노드들로, 이 노드들은 해당 위치에 대응되는 문자열이 트라이가 표현하는 집합에 포함돼있음을 나타낸다.  
트라이의 루트에서 한 노드까지 내려가는 경로에서 만나는 글자들을 모으면 해당 노드에 대응되는 접두사를 얻을 수 있다!  
따라서 각 노드에는 대응되는 문자열을 저장할 필요가 없다.  
트라이의 한 노드를 표현하는 객체는 자손 노드들을 가리키는 포인터 목록과, 이 노드가 종료 노드인지를 나타내는 불린 값 변수로 구성된다.  
자손 노드들을 가리키는 포인터 목록은 동적 배열로 구현하는 것이 아니라,  
입력에 등장할 수 있는 모든 문자에 각각 대응되는 원소를 갖는 **고정 길이 배열로 구현**한다.  
예를 들어 알파벳 대문자로만 구성된 문자열을 저장하는 트라이의 각 노드는 26개짜리 포인터 배열을 갖고 있고,  
배열의 0번 원소는 대응되는 문자열의 맨 뒤에 글자 A를 붙일 경우 얻을 수 있는 문자열 노드의 포인터를 나타낸다.  
해당 노드가 없다면 NULL을 저장한다.  
이와 같은 구조는 **메모리를 엄청나게 낭비**하지만 다음 노드를 찾는 과정에서 어떤 탐색도 필요하지 않은 장점이 있다.  

트라이의 노드를 표현하는 객체의 구현

    // 알파벳 대문자를 저장하는 경우 각 노드는 26개의 자손을 가질 수 있다.
    const int ALPHABETS = 26;
    // 주어진 문자를 [0, ALPHABET-1] 범위의 숫자로 변환해 줌.
    // 출현 문자가 달라지는 경우 이 부분만 바꾸면 된다.
    int toNumber(char ch) { return ch - 'A'; }
    // 트라이의 한 노드를 나타내는 객체
    struct TrieNode {
        TrieNode* children[ALPHABETS];
        // 이 노드가 종료 노드인가?
        bool terminal;
        // 생성자!!!! 그냥 변수는 괄호 {} 앞에서 초기화 해준다.
        TrieNode() : terminal(false) {
            memset(children, 0, sizeof(children));
        }
        // 소멸자!!!!
        ~TrieNode() {
            for (int i = 0; i < ALPHABETS; i++) 
                if (children[i])
                    delete children[i];
        }
        // 이 노드를 루트로 하는 트라이에 문자열 key를 추가한다.
        void insert(const char* key) {
            // 문자열이 끝나면 terminal만 참으로 바꾸고 종료
            if (*key == 0)
                terminal = true;
            else {
                int next = toNumber(*key);
                // 해당 자식 노드가 없다면 생성한다.
                if (children[next] == NULL)
                    children[next] = new TrieNode();
                // 해당 자식 노드로 재귀 호출
                children[next]->insert(key + 1);
            }
        }
        // 이 노드를 루트로 하는 트라이에 문자열 key와 대응되는 노드를 찾는다.
        // 없으면 NULL 반환
        TrieNode* find(const char* key) {
            if (*key == 0) return this;
            int next = toNumber(*key);
            if (children[next] == NULL) return NULL;
            return children[next]->find(key + 1);
        }
    };
                
find()는 찾아낸 문자열에 대응되는 노드가 종료 노드인지 아닌지에 대해서는 신경쓰지 않는다.  
따라서 트라이가 표현하는 집합에 해당 문자열이 진짜로 존재하는지 확인하기 위해서는 반환된 노드의 terminal 멤버가 참인지 확인해야한다.  
하지만 종료 노드가 아닌 노드도 상관없이 반환한다는 것은 장점이기도 하다.  
트라이는 포함하는 문자열들의 모든 접두사에 대응되는 노드들을 가지고 있기 때문에,  
전체 문자열이 아닌 문자열의 첫 일부만을 가지고도 이 문자열로 시작하는 키가 있는지 쉽게 확인할 수 있다.  
이와 같은 속성이 문자열 자동 완성 구현에도 사용됨.  
find()와 insert()는 모두 문자열의 길이만큼 재귀 호출을 수행하기 때문에, 모두 문자열의 길이 M에 선형 비례한다.  
트라이에 포함된 다른 문자열의 개수와는 아무 상관이 없기 때문에 이와 같은 탐색 속도는 다른 어떤 자료구조보다 빠르다.  
하지만 가장 큰 문제점은 필요 공간이 너무 크다는 것. 보통 문제에서는 다루는 문자열 개수가 그렇게 많지 않은 경우로 제한됨.  

In [22]:
# 파이썬 구현. 소멸자 __del__(self)는 필요 없다.
ALPHABETS = 26

toNumber = lambda x: ord(x) -  65
toString = lambda x: chr(x + 65)

class TrieNode():
    
    def __init__(self):
        self.terminal = False
        self.children = [None] * ALPHABETS
    
    # 포인터가 없기 때문에 현재 key의 몇번째 원소를 보고 있는지 알려주는 index i가 추가로 필요.
    def insert(self, key, i=0):
        if i == len(key):
            self.terminal = True
        else:
            next_ = toNumber(key[i])
            if self.children[next_] == None:
                self.children[next_] = TrieNode()
            self.children[next_].insert(key, i + 1)
            
    def find(self, key, i=0):
        if i == len(key):
            return self
        next_ = toNumber(key[i])
        if self.children[next_] == None:
            return None
        return self.children[next_].find(key, i + 1)       
    
    # 현재 노드에 이어서 저장된 문자열들을 모두 출력
    def show(self, string=[]):
        if self.terminal == True:
            print(''.join(string))
            
        for i, v in enumerate(self.children):
            if v:
                string.append(toString(i))
                v.show(string)
                string.pop()
                

In [23]:
root = TrieNode()
root.insert('HELLO')
root.insert('HI')
root.insert('HEY')
root.insert('HOTEL')
root.insert('HELL')
root.insert('HELP')
root.insert('HIS')
root.insert('HIP')
root.insert('HINT')
root.insert('HE')
root.insert('SHE')
root.insert('SHY')
root.insert('SKY')
root.insert('SKI')
root.insert('SHAKE')
root.insert('SHAME')
root.insert('SHALL')

In [24]:
root.show()

HE
HELL
HELLO
HELP
HEY
HI
HINT
HIP
HIS
HOTEL
SHAKE
SHALL
SHAME
SHE
SHY
SKI
SKY


In [26]:
print(bool(root.find('HELLP')))
print(bool(root.find('HELP')))

False
True


#### 사전 자료 구조로 쓰기
terminal 멤버 변수를 불린 값에서 다른 변수형으로 바꾸면 트라이를 사전 자료 구조로 이용할 수 있다.  
예를 들어 terminal을 정수형으로 바꾸면 간단하게 문자열을 정수로 대응하는 사전 자료 구조가 되어 map<string, int>를 대체할 수 있다.  

#### 접미사 트리(Suffix Tree)
트라이의 흥미로운 사용 예로, 여러 문자열들을 트라이에 집어넣는 대신 한 문자열 S의 모든 접미사를 트라이에 집어넣을 수 있다.  
이것을 **접미사 트라이(suffix trie)**라고 부른다.  
문자열 "BANANAS"가 있으면 이 문자열의 접미사는  
`S, AS, NAS, ANAS, NANAS, ANANAS, BANANAS`로 총 7개가 있다.  
문자열의 모든 부분 문자열은 결국 어떤 접미사의 접두사이고 트라이는 저장된 문자열의 모든 접두사들을 저장하기 떄문에,  
접미사 트라이는 이 문자열의 모든 부분 문자열에 대응되는 노드를 갖고 있게 된다.  
예를 들어 부분 문자열 "ANAN"은 "ANANAS"의 접두사이기 때문에 트라이 상에 존재한다.  
따라서 접미사 트라이에서의 검색을 이용하면 어떤 부분 문자열도 빠르게 찾아낼 수 있다.  
하지만 너무 많은 메모리 사용량으로 원 문자열의 길이가 조금만 길어져도 사용이 어렵다.  
1KB짜리 길이 1000의 원 문자열의 접미사의 길이의 합은 대략 50만으로 알파벳 대문자만 치더라도 50만개의 노드를 가져 100MB에 육박..   
이 문제를 해결하기 위해 고안된 것이 접미사 트리(suffix tree)이다.  
접미사 트라이의 많은 부분이 분기 없이 일자로 구성되어 있다는 데 착안하여  
각 간선이 문자열의 한 글자가 아니라 여러 글자에 대응되게 한다.  
접미사 트리의 각 간선은 원 문자열의 일부에 대응되며, 이것은 해당 부분 문자열의 첫 글자와 마지막 글자의 위치로 표현할 수 있다.  
이렇게 바꾸면 전체 노드의 개수는 최대 O(N)이 됨.  
하지만 접미사 트리를 만드는 기본 알고리즘은 $O(N^2)$이고 그보다 효율적인 알고리즘은 너무 복잡해서 잘 쓰이지 않는다.  
보통 접미사 배열을 대신 사용하는 경우가 많다.